In [1]:
import numpy as np
import glob
import datetime
import math
import random
import os
import shutil
import matplotlib.pyplot as plt
import pretty_midi
from pypianoroll import Multitrack, Track
import librosa.display
from utils import *
import write_midi

midi_type = "jazz"
ROOT_PATH = 'D:\Anaconda\Jupyter Notebook\pretty_midi\dataset\MIDI'
test_ratio = 0.1
LAST_BAR_MODE = 'remove'

In [2]:
def get_bar_piano_roll(piano_roll):
    if int(piano_roll.shape[0] % 64) is not 0:
        if LAST_BAR_MODE == 'fill':
            piano_roll = np.concatenate((piano_roll, np.zeros((64 - piano_roll.shape[0] % 64, 128))), axis=0)
        elif LAST_BAR_MODE == 'remove':
            piano_roll = np.delete(piano_roll,  np.s_[-int(piano_roll.shape[0] % 64):], axis=0)
    piano_roll = piano_roll.reshape(-1, 64, 128)
    return piano_roll

In [4]:
"""1. convert_clean.py"""

'1. convert_clean.py'

In [43]:
"""2. choose the clean midi from original sets"""
if not os.path.exists(os.path.join(ROOT_PATH, '{}/cleaner_midi'.format(midi_type))):
    os.makedirs(os.path.join(ROOT_PATH, '{}/cleaner_midi'.format(midi_type)))

l = [f for f in os.listdir(os.path.join(ROOT_PATH, '{}/cleaner'.format(midi_type)))]
print(len(l))
for i in l:
#     print(os.path.join(ROOT_PATH, '{}/{}'.format(midi_type, midi_type), os.path.splitext(i)[0] + '.mid'))
    shutil.copy(os.path.join(ROOT_PATH, '{}/{}'.format(midi_type, midi_type), os.path.splitext(i)[0] + '.mid'),
                os.path.join(ROOT_PATH, '{}/cleaner_midi'.format(midi_type), os.path.splitext(i)[0] + '.mid'))

874


In [44]:
"""3. divide the original set into train and test sets"""
if not os.path.exists(os.path.join(ROOT_PATH, '{}/midi_test'.format(midi_type))):
    os.makedirs(os.path.join(ROOT_PATH, '{}/midi_test'.format(midi_type)))
if not os.path.exists(os.path.join(ROOT_PATH, '{}/midi_train'.format(midi_type))):
    os.makedirs(os.path.join(ROOT_PATH, '{}/midi_train'.format(midi_type)))

l = [f for f in os.listdir(os.path.join(ROOT_PATH, '{}/cleaner_midi'.format(midi_type)))]
print(len(l))
idx_test = np.random.choice(len(l), int(test_ratio * len(l)), replace=False)
print(len(idx_test))
for i in range(len(l)):
    if i in idx_test:
        shutil.move(os.path.join(ROOT_PATH, '{}/cleaner_midi'.format(midi_type), l[i]),
                    os.path.join(ROOT_PATH, '{}/midi_test'.format(midi_type), l[i]))
    else:
        shutil.move(os.path.join(ROOT_PATH, '{}/cleaner_midi'.format(midi_type), l[i]),
                    os.path.join(ROOT_PATH, '{}/midi_train'.format(midi_type), l[i]))

874
87


In [1]:
"""4. merge and crop"""
"""merge different tracks and crop the data to (#, 64, 84, 1)"""
train_test = 'train'
if not os.path.exists(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test))):
    os.makedirs(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test)))
    
l = [f for f in os.listdir(os.path.join(ROOT_PATH, '{}/midi_{}'.format(midi_type, train_test)))]
# print(l)
count = 0
for i in range(len(l)):
    try:
        multitrack = Multitrack(beat_resolution=4, name=os.path.splitext(l[i])[0])
        x = pretty_midi.PrettyMIDI(os.path.join(ROOT_PATH, '{}/midi_{}'.format(midi_type, train_test), l[i]))
        multitrack.parse_pretty_midi(x)

        category_list = {'Piano': [], 'Drums': []}
        program_dict = {'Piano': 0, 'Drums': 0}

        for idx, track in enumerate(multitrack.tracks):
            if track.is_drum:
                category_list['Drums'].append(idx)
            else:
                category_list['Piano'].append(idx)
        tracks = []
        merged = multitrack[category_list['Piano']].get_merged_pianoroll()
#         merged = multitrack[category_list['Piano']].get_merged_pianoroll(mode='max')
        print(merged.shape)

        pr = get_bar_piano_roll(merged)
        print(pr.shape)
        pr_clip = pr[:, :, 24:108]
        print(pr_clip.shape)
#         if int(pr_clip.shape[0] % 4) != 0:
#             pr_clip = np.delete(pr_clip, np.s_[-int(pr_clip.shape[0] % 4):], axis=0)
        pr_re = pr_clip.reshape(-1, 64, 84, 1)
        pr_clip = np.delete(pr_clip, np.s_[-int(pr_clip.shape[0] % 4):], axis=0)
        print(pr_re.shape)
#         if pr_re.shape[0] == 0:
#             continue
#         save_midis(pr_re, os.path.join(ROOT_PATH, 'MIDI/pop/pop_test/cleaner_midi_gen', os.path.splitext(l[i])[0] +
#                                        '.mid'))
        np.save(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test), os.path.splitext(l[i])[0] + '.npy'), pr_re>0)
    except:
        count += 1
        print('Wrong', l[i])
        continue
print(count)

NameError: name 'os' is not defined

In [36]:
"""5. concatenate into a big numpy array file"""
train_test = 'train'
l = [f for f in os.listdir(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test)))]
# print(l)
train = np.load(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test), l[0]))
print(train.shape, train.nonzero()[0].shape)
for i in range(1, len(l)):
    print(i, l[i])
    t = np.load(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test), l[i]))
    train = np.concatenate((train, t), axis=0)
print(train.shape)
np.save(os.path.join(ROOT_PATH, '{}/{}_{}_piano.npy'.format(midi_type, midi_type, train_test)), train)

(20, 64, 84, 1) (3998,)
(20, 64, 84, 1)


In [37]:
"""6. separate numpy array file into single phrases"""
train_test = 'train'
if not os.path.exists(os.path.join(ROOT_PATH, '{}/data_{}'.format(midi_type, train_test))):
    os.makedirs(os.path.join(ROOT_PATH, '{}/data_{}'.format(midi_type, train_test)))
x = np.load(os.path.join(ROOT_PATH, '{}/{}_{}_piano.npy'.format(midi_type, midi_type, train_test)))
print(x.shape)
count = 0
for i in range(x.shape[0]):
    # filter too sparse files
    if x[i].nonzero()[0].shape[0] > 1:
        print(x[i].shape, x[i].nonzero()[0].shape[0], np.max(x[i]))
        count += 1
        data = x[i].reshape(64, 84, 1)
        print(data.shape)
        np.save(os.path.join(ROOT_PATH, '{}/data_{}/{}_piano_{}_{}.npy'.format(midi_type, train_test, midi_type, train_test, count)), data)
#         print(x[i].shape)
   # if count == 11216:
   #     break
print(count)

(20, 64, 84, 1)
(64, 84, 1) 14 True
(64, 84, 1)
(64, 84, 1) 123 True
(64, 84, 1)
(64, 84, 1) 249 True
(64, 84, 1)
(64, 84, 1) 226 True
(64, 84, 1)
(64, 84, 1) 76 True
(64, 84, 1)
(64, 84, 1) 228 True
(64, 84, 1)
(64, 84, 1) 42 True
(64, 84, 1)
(64, 84, 1) 83 True
(64, 84, 1)
(64, 84, 1) 125 True
(64, 84, 1)
(64, 84, 1) 258 True
(64, 84, 1)
(64, 84, 1) 239 True
(64, 84, 1)
(64, 84, 1) 303 True
(64, 84, 1)
(64, 84, 1) 277 True
(64, 84, 1)
(64, 84, 1) 173 True
(64, 84, 1)
(64, 84, 1) 188 True
(64, 84, 1)
(64, 84, 1) 193 True
(64, 84, 1)
(64, 84, 1) 225 True
(64, 84, 1)
(64, 84, 1) 321 True
(64, 84, 1)
(64, 84, 1) 211 True
(64, 84, 1)
(64, 84, 1) 444 True
(64, 84, 1)
20


In [140]:
"""5. concatenate into a big numpy array file"""
train_test = 'train'
l = [f for f in os.listdir(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test)))]
# print(l)
train = np.load(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test), '1_origin.npy'))
print(train.shape, train.nonzero()[0].shape)
for i in range(2, len(l)+1):
    song_name = str(i) + '_origin.npy'
    print(i, song_name)
    t = np.load(os.path.join(ROOT_PATH, '{}/cleaner_npy_{}'.format(midi_type, train_test), song_name))
    train = np.concatenate((train, t), axis=0)
print(train.shape)
np.save(os.path.join(ROOT_PATH, '{}/{}_{}_piano.npy'.format(midi_type, midi_type, train_test)), train)

(1, 64, 84, 1) (14,)
2 2_origin.npy
3 3_origin.npy
4 4_origin.npy
5 5_origin.npy
6 6_origin.npy
7 7_origin.npy
8 8_origin.npy
9 9_origin.npy
10 10_origin.npy
11 11_origin.npy
12 12_origin.npy
13 13_origin.npy
14 14_origin.npy
15 15_origin.npy
16 16_origin.npy
17 17_origin.npy
18 18_origin.npy
19 19_origin.npy
20 20_origin.npy
(20, 64, 84, 1)


In [141]:
"""test for writ midi files"""
midi_type = 'jazz'
train_test = 'train'
name='topgrooves'
# l = [f for f in os.listdir(os.path.join(ROOT_PATH, '{}/data_{}'.format(midi_type, train_test)))]

# for i in range(len(l)):
# #    print(l[i])
t = np.load(os.path.join(ROOT_PATH, '{}/{}_{}_piano.npy'.format(midi_type, midi_type, train_test)))
save_midis(t,os.path.join(ROOT_PATH, '{}/new_songs/{}.mid'.format(midi_type, name)))

In [24]:
a=[np.array([1,2,3])]
a.append(np.array([4,4,4]))
b = np.squeeze(np.array(a))
# print(np.nonzero(b)[0].shape[0])
# print(np.where(b > 0)[0].shape[0])
d = np.zeros((1,3))
print(b)

[[1 2 3]
 [4 4 4]]
